In [1]:
from __future__ import division
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from tensorflow.contrib.layers import fully_connected
from tensorflow.contrib.layers import dropout
from tensorflow.contrib.layers import batch_norm

In [2]:
#load data
if os.name=='nt':
    parentDir= 'C:/Users/Administrator.NBJXUEJUN-LI/Desktop/project/ncpltnpj/'
else:
    parentDir= '/Users/snakepointid/Documents/project/cpltnpj/'
UserTargdayLabel =pd.read_csv(parentDir+'save/DNN_feeds.csv',header=0,encoding='gbk' )

In [3]:
 #perpare datas
predData  = UserTargdayLabel[UserTargdayLabel['label']==-1]
trainData = UserTargdayLabel[UserTargdayLabel['labelday']!='2016-04-11']
testData  = UserTargdayLabel[UserTargdayLabel['labelday']=='2016-04-11']

In [4]:
trainX     = np.array(trainData.ix[:,3:],dtype=np.float32)
trainy     = np.array(trainData.ix[:,2],dtype=np.float32)
trainy     = trainy.reshape((len(trainy),1))

testX      = np.array(testData.ix[:,3:],dtype=np.float32)
testy      = np.array(testData.ix[:,2],dtype=np.float32)
testy      = testy.reshape((len(testy),1))

predX      = np.array(predData.ix[:,3:],dtype=np.float32)

In [5]:
trainy.sum()/len(trainy),testy.sum()/len(testy)

(0.088514128393867131, 0.010754054767468007)

In [6]:
#set neurons' dimemsions
n_inputs  = len(trainX[0])
n_hidden1 = 500
n_hidden2 = 300
n_hidden3 = 200
n_hidden4 = 100
n_outpus  = 1

In [7]:
is_training = tf.placeholder_with_default(False, shape=(), name='is_training') 
keep_prob = 0.5
bn_params={
    'is_training':is_training,
    'decay':0.99,
    'updates_collections':None
}
with tf.name_scope("dnn"):  
    X = tf.placeholder(tf.float32, shape=[None, n_inputs])
    y = tf.placeholder(tf.float32, shape=[None, n_outpus])
    X_drop = dropout(X, keep_prob, is_training=is_training)
    hidden1 = fully_connected(X_drop,  n_hidden1, scope="hidden1",activation_fn=tf.nn.elu,normalizer_fn=batch_norm,normalizer_params=bn_params)
    hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2",activation_fn=tf.nn.elu,normalizer_fn=batch_norm,normalizer_params=bn_params)
    hidden3 = fully_connected(hidden2, n_hidden3, scope="hidden3",activation_fn=tf.nn.elu,normalizer_fn=batch_norm,normalizer_params=bn_params)
    hidden4 = fully_connected(hidden3, n_hidden4, scope="hidden4",activation_fn=tf.nn.elu,normalizer_fn=batch_norm,normalizer_params=bn_params)
    logits  = fully_connected(hidden4, n_outpus,  scope="outputs",activation_fn=None)
    actlogits = tf.sigmoid(logits)

In [8]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [9]:
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
init = tf.global_variables_initializer()
n_epochs = 3
batch_size = 10000
batch_num = len(trainX) // batch_size+1
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):  
        for batch in range(batch_num):
            X_batch, y_batch = trainX[batch*batch_size:(batch+1)*batch_size],trainy[batch*batch_size:(batch+1)*batch_size]
            sess.run(training_op, feed_dict={X: X_batch,y:y_batch,is_training:True})
         
        train_out = logits.eval(feed_dict={X: trainX,y:trainy})
        test_out  = logits.eval(feed_dict={X: testX, y:testy})
        print (roc_auc_score(trainy,train_out),roc_auc_score(testy,test_out))
    test_out  = actlogits.eval(feed_dict={X: testX, y:testy})

In [ ]:
from matplotlib import pyplot as plt
def plotF1score(score,y):
    scorePs = np.arange(100)/100
    F1s = np.zeros(100,dtype=np.float32)
    for i in range(100):   
        scoreP = scorePs[i]
        TP = np.equal(y[score>scoreP],1).astype('f').sum()
        pred_posi_num = (score>scoreP).astype('f').sum()
        all_posi_num  = y.sum()
        Recall  = TP/all_posi_num
        Precise = TP/pred_posi_num
        F1s[i] = 6*Recall*Precise/(5*Recall+Precise)
    plt.plot(scorePs,F1s)
    plt.show()

In [ ]:
test_out

In [ ]:
plotF1score(test_out,testy)